# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd # импортируем библиотеку

**Для начала я открою нашу таблицу и изучу содержимое.**

In [2]:
data = pd.read_csv('/datasets/data.csv') # открываем таблицу
data # выводим ее на экран

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Шаг 2.1 Заполнение пропусков

**Даже просто при визуальном осмотре я заметил множество ошибок, разберемся сначала с пропусками!**

In [3]:
data.isna().sum() # смотрим количество пропущенных значений.

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data[data['days_employed'].isna()].head() # узнаем, какие значения у нас вместо пропусков.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [5]:
days_employed1 = data['days_employed'].median() # ищем медиану столбца `days_employed` 
data['days_employed'] = data['days_employed'].fillna(days_employed1) # заполняем наши пропуски медианным значением
total_income1 = data['total_income'].median() # ищем медиану столбца `total_income`
data['total_income'] = data['total_income'].fillna(total_income1) # заполняем наши пропуски медианным значением


Убедимся, сработал ли наш способ заполнения пропусков.

In [6]:
data.isna().sum() # смотрим количество пропущенных значений.

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Действительно, теперь все пропуски в нашей таблице отсутствуют, можно двигаться дальше!

### Шаг 2.2 Проверка данных на аномалии и исправления.

**Видно, что в столбце `days_employed` много отрицательных значений, чего на самом деле не может быть, исправим это дело.**

In [7]:
data['days_employed'] = data['days_employed'].abs() # убираем знак минус из всех значений

Проверим, получилось ли у нас.

In [8]:
data['days_employed'] # посмотрим столбец `days_employed`

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Да, все отлично! Скорее всего, данная ошибка произошла из-за технической неисправности.

### Шаг 2.3. Изменение типов данных.

**Изменим вещественный тип данных на целочисленный.**

In [9]:
data['total_income'] = data['total_income'].astype('int') # изменяем тип на целочисленный

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21369 entries, 0 to 21470
Data columns (total 13 columns):
children                 21369 non-null int64
days_employed            21369 non-null float64
dob_years                21369 non-null int64
education_id             21369 non-null int64
family_status_id         21369 non-null int64
gender                   21369 non-null object
income_type              21369 non-null object
debt                     21369 non-null int64
total_income             21369 non-null int64
purpose                  21369 non-null object
total_income_category    21369 non-null object
purpose_category         21369 non-null object
salary_cat               21369 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.3+ MB


### Шаг 2.4. Удаление дубликатов.

**Найдем количество строк-дубликатов в нашей таблице.**

In [10]:
data.duplicated().sum() # проверка на строки-дубликаты.

54

Мы видим, что в нашей таблице присутствуют строки-дубликаты, уберем их.

In [11]:
data = data.drop_duplicates().reset_index(drop=True) # удаление строк-дубликатов и нумерация строк заново.

Проверим, есть ли теперь строки-дубликаты.

In [12]:
data.duplicated().sum() # проверка на строки-дубликаты.

0

Мы видим, что строк-дубликатов больше нет, теперь займемся неявными дубликатами!

In [13]:
data['education'] = data['education'].str.lower() #переводим столбец в нижний регистр

Проверим, какие теперь значения находятся в столбце `education`

In [14]:
data['education'].head() # смотрим наш столбец

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: object

Как мы можем заметить, все значения теперь приведены к нижнему регистру. Проверим остальные столбцы.

In [15]:
data['family_status'].unique() # проверка на неявные дубликаты.

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [16]:
data['children'].unique() 

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видим, что есть значение -1, скорее всего перепутали знак, изменим его, пока не забыли, так же наблюдаем, что у кого-то аж 20 детей, что можно сказать, удачи.

In [17]:
data['children'] = data['children'].abs() # убираем отрицательные значения из столбца.

In [18]:
data['children'].unique() 

array([ 1,  0,  3,  2,  4, 20,  5])

Замена прошла успешно, идем дальше.

In [19]:
data['days_employed'].unique() 

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

In [20]:
data['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Видим, что есть значение 0, такого не может быть, удаляем строку.

In [ ]:
data = data.loc[data['dob_years'] != 0]

In [22]:
data['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [23]:
data['education_id'].unique() 

array([0, 1, 2, 3, 4])

In [24]:
data['family_status_id'].unique() 

array([0, 1, 2, 3, 4])

In [25]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Видим лишнее значение 'XNA', такого пола не существует, убираем!

In [26]:
data = data.loc[data['gender'] != 'XNA'] # удаляем лишнюю строчку

In [27]:
data['gender'].unique()

array(['F', 'M'], dtype=object)

Теперь все хорошо!

In [28]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [29]:
data['debt'].unique()

array([0, 1])

In [30]:
data['total_income'].unique()

array([253875, 112080, 145885, ...,  89672, 244093,  82047])

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Мы видим страшную вещь, цели получения кредита ужасно записаны, но эту ошибку мы исправим чуть позже. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

**Создадим два датафрейма и перенесем в них заданные столбцы!**

In [32]:
data_education = data[['education_id', 'education']] # создаем новые датафреймы с определенными значениями
data_status = data[['family_status_id', 'family_status']]

Удалим столбец `education` И `family_status` из нашей таблицы.

In [33]:
del data['education'] # удаляем ненужные столбцы
del data['family_status']

In [34]:
data # проверим выполнения задачи

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

In [35]:
def total_income_category(income): # создаем функцию и задаем условия
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    return 'A'


In [36]:
data['total_income_category'] = data['total_income'].apply(total_income_category) # создаем столбец в нашей таблице

In [37]:
data # проверка выполнения задачи

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

In [38]:
def purpose_category(purpose): # создаем функцию, которая будет обрабатывать информацию за нас
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    elif 'жиль' or 'недвижим' in purpose:
        return 'операции с недвижимостью'

In [39]:
data['purpose_category'] = data['purpose'].apply(purpose_category) # создаем столбец в нашей таблице

In [40]:
data # проверка выполнения задачи

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


In [41]:
data['purpose_category'].unique() # еще одна проверка на значения

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Ответы на вопросы.

# Вопрос 1. Есть ли зависимость между количеством детей и возвратом кредита в срок? <a id='intro'></a>

In [42]:
data_pivot_children = data.pivot_table(index='children', values='debt', aggfunc=['sum', 'count']) # создаем сводную таблицу
data_pivot_children.reset_index()
data_pivot_children['debt_%'] = data_pivot_children['sum'] / data_pivot_children['count'] * 100 # находим процент задолжностей
display(data_pivot_children)

,sum,count,debt_%
,debt,debt,
children,,,
0,1058,14037,7.537223
1,442,4840,9.132231
2,194,2039,9.514468
3,27,328,8.231707
4,4,41,9.756098
5,0,9,0.000000
20,8,75,10.666667


Заемщики, не имеющие детей, менее склонны к просрочке по выплатам кредита.

# Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?<a id='intro'></a>

In [43]:
data_pivot_family = data.pivot_table(index='family_status_id', columns='debt', values='gender', aggfunc = 'count') # создаем сводную таблицу
data_pivot_family['count_debt'] = data_pivot_family[0] + data_pivot_family[1] # выражаем каждый столбец
data_pivot_family.insert(1, 'dept_0 %', data_pivot_family[0] / data_pivot_family['count_debt'] * 100)
data_pivot_family.insert(3, 'dept_1 %', data_pivot_family[1] / data_pivot_family['count_debt'] * 100)
data_pivot_family.merge(data_status, on='family_status_id', how='left') # объединяю несколько таблиц в одну

,family_status_id,0,dept_0 %,1,dept_1 %,count_debt,family_status
0,0,11368,92.460350,927,7.539650,12295,женат / замужем
1,0,11368,92.460350,927,7.539650,12295,женат / замужем
2,0,11368,92.460350,927,7.539650,12295,женат / замужем
3,0,11368,92.460350,927,7.539650,12295,женат / замужем
4,0,11368,92.460350,927,7.539650,12295,женат / замужем
...,...,...,...,...,...,...,...
21364,4,2521,90.229062,273,9.770938,2794,Не женат / не замужем
21365,4,2521,90.229062,273,9.770938,2794,Не женат / не замужем
21366,4,2521,90.229062,273,9.770938,2794,Не женат / не замужем
21367,4,2521,90.229062,273,9.770938,2794,Не женат / не замужем


In [44]:
data_pivot_family['dept_0 %'] # посмотрим проценты к каждому статусу

family_status_id
0    92.460350
1    90.678580
2    93.501048
3    92.827004
4    90.229062
Name: dept_0 %, dtype: float64

Граждане, не имеющие узаконенных отношений, более склонны к просрочке по выплатам кредита.

# Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<a id='intro'></a>

In [45]:
def salary_cat(row): # создаем функцию, для создания нового столбика
    if row['total_income'] <= 50000:
        return 'бедный'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 1000000:
        return 'зажиточный'
    else:
        return 'миллионер'

In [46]:
data['salary_cat'] = data.apply(salary_cat, axis=1) # создаем новый столбец в нашей таблице

In [47]:
data # проверка

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,salary_cat
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,зажиточный
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,средний
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,зажиточный
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,зажиточный
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,зажиточный
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,зажиточный
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,зажиточный
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,средний
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем,зажиточный


In [48]:
def relation(category): # создаем функцию, которая будем считать % задолжностей для каждого статуса зарплат
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

In [49]:
relation('salary_cat') # запускаем нашу функцию

,debt
salary_cat,
бедный,0.062162
миллионер,0.080000
зажиточный,0.081384
средний,0.081547


Удивительно, но заемщики с уровнем дохода < 50000р. менее склонны нарушать обязательства по выплатам кредита.

# Вопрос 4. Как разные цели кредита влияют на его возврат в срок?<a id='intro'></a>

В прошлом задании я создал отличную функцию, которая поможет ответить на вопрос и сейчас.

In [50]:
relation('purpose_category')

,debt
purpose_category,
операции с недвижимостью,0.072357
проведение свадьбы,0.079276
получение образования,0.092593
операции с автомобилем,0.093327


Мы видим, что люди, берущие кредиты для операций с недвижимостью более ответственны чем остальные и стараются не нарушать обязательства по выплатам.

## Общий вывод:

При подготовке материалов в данные были внесены изменения в целях корректной интерпретации показателей:

* Заполнены пропуски и аномальные данные медианными значениями, 
* Переведены в положительные значения отрицательные аномалии,
* Исключены полные и неявные дубликаты.

**Зависимость между количеством детей и возвратом кредита в срок.**

* Семьи с 5 детьми имеют больше ответственности и как результат отсутствие задолженность,
*  Бездетные семьи наиболее широкая группа заемщиков с наименьшей задолженность,
* Семьи с 3 детьми более подвержены риску просрочки задолженности,
* Наибольшая группа риска просрочки задолженности семьи с 1 и 2 ребенком,
* Семьи с 4 детьми самая высокая и самая малочисленная группа риска просрочки задолженности.

**Заемщики находящиеся в браке/планирующие брак, с долгосрочным кредитом или кредитом на свадьбу, в не зависимости от уровня дохода - являются лучшими клиентами банка.**

**Заемщики вне брака, с кредитом на автомобиль или образование, с 1-2 или 4 детьми - - являются худшими клиентами банка.**